In [1]:
import os
import glob
import json
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../har_research")

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import clip
from PIL import Image
from IPython.display import display

In [ ]:
from har_research.image_scan import iter_image_entries

In [3]:
model, preprocess = clip.load("ViT-B/32")

## get image features

In [ ]:
def store_har_image_features(glob_pattern: str, feature_filename: str):
    with open(feature_filename, "w") as fp:
        for e, image in iter_image_entries(glob_pattern):
            if image.size[0] < 10:
                continue
            try:
                image = preprocess(image).unsqueeze(0).cuda()
                with torch.no_grad():
                    features = model.encode_image(image)
            except Exception as ex:
                print(e["request"]["url"], ":", ex)
                continue

            line = '{"url":"%s","features":[%s]}\n' % (
                e["request"]["url"],
                ",".join(str(f)[:8] for f in features.cpu().numpy()[0])
            )
            #print(line)
            #break
            fp.write(line)

    #print(e["request"]["url"])
    
store_har_image_features(
    "../har_research/automatic/recordings/*/*.json",
    "clip-image-features.ndjson"
)

## get text features

In [4]:
torch.cuda.empty_cache()

In [5]:
things = [
    "CIA", "LSD",
    "Charles Manson", "Robert Anton Wilson", "Timothy Leary", "Ken Kesey", "Bob Dobbs",
    "James Joyce", "Thomas Pynchon", "Steven Erikson", "Hermann Hesse",
    "John F. Kennedy", "George Bush", "Barack Obama", "Donald Trump", "Gerhard Schröder", "Angela Merkel",
    "a cat", "a dog", "a whale", "a spider", "a penguin", "a tree", "a car", "a kid",
    "computer", "a super-human intelligent computer",
    "a naked woman", "a naked man",
]
adjectives = [
    "",
    "a huge gigantic~", "a tiny little~",
    "a scared and frightened~", "a curious and lively~",
    "a compassionate and helpful~",
    "a strange and incomprehensible~",
    "the complete opposite of",
]
prefixes = [
    "",
    "a photo of", "a drawing of", "a painting of", "a sketch of", "a high-professional representation of", 
    "a statue of", "the anatomy of", "a description of",
    "a dream of", "a surrealistic version of",
    "close-up of", "rear-view of", "top-view of",
]

labels = []
for thing in things:
    for adjective in adjectives:
        for prefix in prefixes:
            the_thing = thing
            if adjective.endswith("~") and thing.startswith("a "):
                the_thing = thing[2:]
            labels.append(f"{prefix}/{adjective.rstrip('~')}/{thing}")
    
print(len(labels))
labels

3248


['//CIA',
 'a photo of//CIA',
 'a drawing of//CIA',
 'a painting of//CIA',
 'a sketch of//CIA',
 'a high-professional representation of//CIA',
 'a statue of//CIA',
 'the anatomy of//CIA',
 'a description of//CIA',
 'a dream of//CIA',
 'a surrealistic version of//CIA',
 'close-up of//CIA',
 'rear-view of//CIA',
 'top-view of//CIA',
 '/a huge gigantic/CIA',
 'a photo of/a huge gigantic/CIA',
 'a drawing of/a huge gigantic/CIA',
 'a painting of/a huge gigantic/CIA',
 'a sketch of/a huge gigantic/CIA',
 'a high-professional representation of/a huge gigantic/CIA',
 'a statue of/a huge gigantic/CIA',
 'the anatomy of/a huge gigantic/CIA',
 'a description of/a huge gigantic/CIA',
 'a dream of/a huge gigantic/CIA',
 'a surrealistic version of/a huge gigantic/CIA',
 'close-up of/a huge gigantic/CIA',
 'rear-view of/a huge gigantic/CIA',
 'top-view of/a huge gigantic/CIA',
 '/a tiny little/CIA',
 'a photo of/a tiny little/CIA',
 'a drawing of/a tiny little/CIA',
 'a painting of/a tiny little/CIA

In [6]:
labels_tokens = clip.tokenize([l.replace("/", " ").replace("  ", " ").strip() for l in labels])
with torch.no_grad():
    labels_features = model.encode_text(labels_tokens.cuda()).cpu().numpy()

In [7]:
df = pd.DataFrame(labels_features)
df.index = labels
df.to_csv("clip-label-features-all-3.csv")
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
//CIA,-0.095825,0.331543,0.234375,0.171509,-0.055115,-0.260254,-0.055084,-0.539062,-0.124512,0.191895,...,-0.028793,0.376465,0.070312,-0.028122,0.112000,-0.052612,-0.048828,0.023117,-0.051178,0.001927
a photo of//CIA,-0.156006,0.302734,0.130737,0.042572,-0.008682,-0.344727,-0.150391,-1.205078,0.052490,0.321045,...,-0.292725,0.255859,0.092529,0.022156,0.155396,-0.100769,-0.088196,-0.087280,0.058136,-0.084961
a drawing of//CIA,0.034149,-0.168579,0.024139,-0.026505,-0.099426,-0.309814,-0.293701,-1.047852,0.190796,0.018082,...,-0.096680,0.227051,0.229004,0.125610,-0.076233,-0.063782,0.145752,-0.356934,0.201416,-0.282227
a painting of//CIA,0.184937,-0.107971,-0.170166,0.166382,0.042694,-0.489990,-0.287842,-0.925781,0.126831,0.151489,...,-0.213135,0.148804,0.499268,-0.124939,0.210815,0.141724,-0.032959,-0.322998,0.182495,-0.178101
a sketch of//CIA,0.111938,-0.232666,0.064270,-0.034119,-0.153564,-0.268066,-0.303955,-1.171875,0.067688,0.001092,...,-0.036530,0.036377,0.259766,0.002300,-0.061981,-0.173706,0.199585,-0.205811,0.196655,-0.260986
